# 3分类，500个样本，20个特征值，共3层，第一层13个神经元，第二层8个神经元

In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F

# 设置随机种子确保实验可重复性
torch.manual_seed(420)

# 1.生成随机数据：
# X: 500个样本，每个样本20维特征（形状[500, 20]）
# y: 500个标签，值在[0, 3)范围内随机整数（3分类问题）
X = torch.rand([500, 20], dtype=torch.float32)  # 输入特征矩阵
y = torch.randint(low=0, high=3, size=[500,], dtype=torch.float32)  # 标签向量

# 获取输入特征维度和输出类别数
input_ = X.shape[1]  # 输入特征维度=20
output_ = len(y.unique())  # 输出类别数=3（因为y∈{0,1,2}）

# 2.定义神经网络模型
class Model(nn.Module):
    def __init__(self, in_features, out_features):
        super().__init__()
        # 定义网络层：
        # 第一层：20维输入 -> 13维输出（无偏置）
        self.linear1 = nn.Linear(in_features, 13, bias=False)
        # 第二层：13维输入 -> 8维输出（无偏置）
        self.linear2 = nn.Linear(13, 8, bias=False)
        # 输出层：8维输入 -> 3维输出（有偏置）
        self.output = nn.Linear(8, out_features, bias=True)
    
    def forward(self, x):
        # 前向传播：
        sigma1 = torch.relu(self.linear1(x))  # 第一层后接ReLU激活
        sigma2 = torch.sigmoid(self.linear2(sigma1))  # 第二层后接Sigmoid激活
        zhat = self.output(sigma2)  # 输出层（无激活函数，直接输出logits）
        return zhat

# 3.实例化模型（固定随机种子保证初始化一致）
torch.manual_seed(420)
net = Model(in_features=input_, out_features=output_)

# 4.前向计算：获取模型预测结果（原始logits）
zhat = net.forward(X)  # 输出形状[500, 3]

# 5.定义损失函数（交叉熵损失，内部自动结合LogSoftmax+NLLLoss）
criterion = nn.CrossEntropyLoss()

# 计算损失（需将y转为long类型，因为CrossEntropyLoss要求标签是整数）
loss = criterion(zhat, y.long())

# 6.反向传播（retain_graph=True保留计算图以便后续可能的重用）
loss.backward(retain_graph=True)

# 打印梯度形状：
# linear1的梯度形状应与权重矩阵相同：[13, 20]
print(net.linear1.weight.grad.shape)  # 输出: torch.Size([13, 20])
# linear2的梯度形状：[8, 13]
print(net.linear2.weight.grad.shape)  # 输出: torch.Size([8, 13])
#7.动量法 v(t)=gamma*v(t-1)-lr*dw ;w(t+1)=w(t)+v(t)
lr=0.1
gamma=0.9
dw=net.linear1.weight.grad
w=net.linear1.weight.data
# t=1,走第一步，进行首次迭代，需要V0
v=torch.zeros(size=[dw.shape[0],dw.shape[1]])
v=gamma*v-lr*dw
w+=v
print(w)

torch.Size([13, 20])
torch.Size([8, 13])
tensor([[ 1.3656e-01, -1.3459e-01,  2.1281e-01, -1.7763e-01, -6.8219e-02,
         -1.5410e-01,  1.7245e-01,  8.3883e-02, -1.1153e-01, -1.7294e-01,
         -1.2947e-01, -4.3139e-02, -1.1413e-01,  1.6294e-01, -9.4083e-02,
         -1.4629e-01, -6.8983e-02, -2.1836e-01, -1.0859e-01, -1.2199e-01],
        [ 4.8174e-02,  1.8190e-01,  2.4149e-02, -1.3026e-01,  9.2083e-02,
         -9.5210e-02, -1.0582e-01, -4.2824e-02, -1.1669e-01,  2.4615e-02,
          1.8153e-01,  3.0533e-02,  1.3506e-01, -1.9422e-01, -1.7593e-01,
         -2.9742e-02,  2.0621e-04,  1.3959e-01, -1.9662e-01,  9.3331e-02],
        [-1.9184e-01,  3.6138e-02,  1.4793e-01,  3.0939e-02,  7.1511e-02,
          1.4233e-01,  2.2135e-01, -1.4023e-01,  7.3449e-02,  1.8421e-01,
          1.2732e-01, -2.0247e-01, -1.5496e-01, -2.1887e-01,  9.9163e-02,
          2.2131e-01, -2.1647e-01,  1.7898e-01, -2.0911e-01, -2.7156e-02],
        [ 1.8145e-01, -3.5160e-02,  2.4802e-02,  1.6301e-01, -1.8755